In [1]:
using Plots
using PyCall
using ControlSystems

gr()

Plots.GRBackend()

In [2]:
zc=0.23
g=9.8
dt=0.0025
L=0.12
l=0.05
x=0.05
ft=0.05 #最初の全脚支持期間

#support time
Tsup=Int32(0.5/dt)
#both foot support point time
Tdl=Int32(0.1/dt)
#preview step
M=400
            #r             c         l
real_sx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
        
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #脚踏みver  rcl

real_sy=[0.0; L/2; L;
    
        0.0; L/2; L;
         0.0;  L/2; L;
         0.0; L/2; L;
        
         0.0; L/2; L; 
         0.0; L/2; L; 
         0.0; L/2; L;
        
         0.0; L/2; L; 
         0.0; L/2; L; 
         0.0; L/2; L;
        
         0.0; L/2; L]

pltsx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #plot用
pltsy=[0.0; L/2; L]

leg_sign=[0;1;2;3;1;2;3;1;2;3;0]#上げる脚とステップの関係

f_zmp_x=abs(real_sx[1]-real_sx[2])/2+0.02
f_zmp_y=L/2

e_zmp_x=abs(real_sx[1]-real_sx[2])/2+0.02
e_zmp_y=L/2

#Virtual support point
sx=[f_zmp_x
    abs(real_sx[5]-real_sx[6])/2 
    real_sx[7]+abs(real_sx[7]-real_sx[9])/2 
    abs(real_sx[10]-real_sx[11])/2
    
    abs(real_sx[5]-real_sx[6])/2 
    real_sx[7]+abs(real_sx[7]-real_sx[9])/2 
    abs(real_sx[10]-real_sx[11])/2
    
    abs(real_sx[5]-real_sx[6])/2 
    real_sx[7]+abs(real_sx[7]-real_sx[9])/2 
    abs(real_sx[10]-real_sx[11])/2
    e_zmp_x]

sy=[f_zmp_y
    real_sy[5]+abs(real_sy[5]-real_sy[6])/2  
    abs(real_sy[7]-real_sy[9])/2  
    abs(real_sy[10]-real_sy[11])/2
    
    real_sy[5]+abs(real_sy[5]-real_sy[6])/2  
    abs(real_sy[7]-real_sy[9])/2  
    abs(real_sy[10]-real_sy[11])/2
    
    real_sy[5]+abs(real_sy[5]-real_sy[6])/2  
    abs(real_sy[7]-real_sy[9])/2  
    abs(real_sy[10]-real_sy[11])/2
    e_zmp_y]

vpoint_sum=length(sx)
full_leg_step_sum=vpoint_sum-1

#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,full_leg_step_sum)
x_tdl_b=zeros(Float64,full_leg_step_sum)
y_tdl_a=zeros(Float64,full_leg_step_sum)
y_tdl_b=zeros(Float64,full_leg_step_sum)

for i in 1:full_leg_step_sum
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:full_leg_step_sum
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=zeros(vpoint_sum*Tsup+M+1 + full_leg_step_sum*Tdl)
py_ref=zeros(px_ref)

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            px_ref[k+step]=sx[i]
        end
    else
        for k in 1:Tsup
            px_ref[k+step]=sx[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            px_ref[s+step]=x_tdl_a[i]*(s+step)*dt+x_tdl_b[i]
        end
        step=step+Tdl
    end
end

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            py_ref[k+step]=sy[i]
        end
    else
        for k in 1:Tsup
            py_ref[k+step]=sy[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            py_ref[s+step]=y_tdl_a[i]*(s+step)*dt+y_tdl_b[i]
        end
        step=step+Tdl
    end
end

#total step
k_max=length(px_ref)-M
#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[20000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]
    0.0
    0.0]

y=[ sy[1]+0.001
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 

xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y

dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR
old_x=x
old_y=y
xu=0
yu=0
P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 -0.131371

In [5]:
for i in 1:k_max-1
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]    
    k+=1
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    dx=x-old_x
    dy=y-old_y
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    old_x=x
    old_y=y
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end

In [6]:
plot(t*dt,px_ref,lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t*dt,x_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t*dt,px,lab="Target ZMP",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 time[ms] 
 
 
 ZMP 
 
 
 x[m] 
 
<polyline clip-path="url(#clip6902)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,122.257 39.5827,122.257 39.7953,122.257 40.0079,122.257 40.2205,122.257 40.4331,122.257 40.6457,122.257 40.8583,122.257 41.0709,122.257 41.2835,122.257 
 41.4961,122.257 41.7087,122.257 41.9213,122.257 42.1339,122.257 42.3465,122.257 42.5591,122.257 42.7717,122.257 42.9843,122.257 43.1969,122.257 43.4094,122.257 
 43.622,122.257 43.8346,122.257 44.0472,122.257 44.2598,122.257 44.4724,122.257 44.685,122.257 44.8976,122.257 45.1102,122.257 45.3228,122.257 45.5354,122.257 
 45.748,122.257 45.9606,122.257 46.1732,122.257 46.3858,122.257 46.5984,122.257 46.811,122.257 47.0236,122.257 47.2362,122.257 47.4488,122.257 47.6614,122.257 
 47.874,122.257 48.0866,122.257 48.2992,122.257 48.5118,122.257 48.7244,122.257 48.937,122.257 49.1496,122.257 49.3622,122.257 49.5748,122.257 49.7874,122.257 
 50,122.257 50.2126,122.257 50.4252,122.257 50.6378,122.257 50.8504,122.257 51.063,122.257 51.2756,122.257 51.4882,122.257 51.7008,122.257 51.9134,122.257 
 52.126,122.257 52.3386,122.257 52.5512,122.257 52.7638,122.257 52.9764,122.257 53.189,122.257 53.4016,122.257 53.6142,122.257 53.8268,122.257 54.0394,122.257 
 54.252,122.257 54.4646,122.257 54.6772,122.257 54.8898,122.257 55.1024,122.257 55.315,122.257 55.5276,122.257 55.7402,122.257 55.9528,122.257 56.1654,122.257 
 56.378,122.257 56.5906,122.257 56.8031,122.257 57.0157,122.257 57.2283,122.257 57.4409,122.257 57.6535,122.257 57.8661,122.257 58.0787,122.257 58.2913,122.257 
 58.5039,122.257 58.7165,122.257 58.9291,122.257 59.1417,122.257 59.3543,122.257 59.5669,122.257 59.7795,122.257 59.9921,122.257 60.2047,122.257 60.4173,122.257 
 60.6299,122.257 60.8425,122.257 61.0551,122.257 61.2677,122.257 61.4803,122.257 61.6929,122.257 61.9055,122.257 62.1181,122.257 62.3307,122.257 62.5433,122.257 
 62.7559,122.257 62.9685,122.257 63.1811,122.257 63.3937,122.257 63.6063,122.257 63.8189,122.257 64.0315,122.257 64.2441,122.257 64.4567,122.257 64.6693,122.257 
 64.8819,122.257 65.0945,122.257 65.3071,122.257 65.5197,122.257 65.7323,122.257 65.9449,122.257 66.1575,122.257 66.3701,122.257 66.5827,122.257 66.7953,122.257 
 67.0079,122.257 67.2205,122.257 67.4331,122.257 67.6457,122.257 67.8583,122.257 68.0709,122.257 68.2835,122.257 68.4961,122.257 68.7087,122.257 68.9213,122.257 
 69.1339,122.257 69.3465,122.257 69.5591,122.257 69.7717,122.257 69.9843,122.257 70.1969,122.257 70.4094,122.257 70.622,122.257 70.8346,122.257 71.0472,122.257 
 71.2598,122.257 71.4724,122.257 71.685,122.257 71.8976,122.257 72.1102,122.257 72.3228,122.257 72.5354,122.257 72.748,122.257 72.9606,122.257 73.1732,122.257 
 73.3858,122.257 73.5984,122.257 73.811,122.257 74.0236,122.257 74.2362,122.257 74.4488,122.257 74.6614,122.257 74.874,122.257 75.0866,122.257 75.2992,122.257 
 75.5118,122.257 75.7244,122.257 75.937,122.257 76.1496,122.257 76.3622,122.257 76.5748,122.257 76.7874,122.257 77,122.257 77.2126,122.257 77.4252,122.257 
 77.6378,122.257 77.8504,122.257 78.063,122.257 78.2756,122.257 78.4882,122.257 78.7008,122.257 78.9134,122.257 79.126,122.257 79.3386,122.257 79.5512,122.257 
 79.7638,122.257 79.9764,122.257 80.189,122.257 80.4016,122.257 80.6142,122.257 80.8268,122.257 81.0394,122.257 81.252,122.257 81.4646,122.257 81.6772,122.257 
 81.8898,123.968 82.1024,125.679 82.315,127.39 82.5276,129.101 82.7402,130.812 82.9528,132.523 83.1654,134.234 83.378,135.945 83.5906,137.655 83.8031,139.366 
 84.0157,141.077 84.2283,142.788 84.4409,144.499 84.6535,146.21 84.8661,147.921 85.0787,149.632 85.2913,151.343 85.5039,153.054 85.7165,154.765 85.9291,156.476 
 86.1417,158.187 86.3543,159.898 86.5669,161.609 86.7795,163.32 86.9921,165.031 87.2047,166

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,py,lab="ZMP y",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 time[ms] 
 
 
 ZMP[m] 
 
<polyline clip-path="url(#clip7002)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,132.48 39.5827,132.48 39.7953,132.48 40.0079,132.48 40.2205,132.48 40.4331,132.48 40.6457,132.48 40.8583,132.48 41.0709,132.48 41.2835,132.48 
 41.4961,132.48 41.7087,132.48 41.9213,132.48 42.1339,132.48 42.3465,132.48 42.5591,132.48 42.7717,132.48 42.9843,132.48 43.1969,132.48 43.4094,132.48 
 43.622,132.48 43.8346,132.48 44.0472,132.48 44.2598,132.48 44.4724,132.48 44.685,132.48 44.8976,132.48 45.1102,132.48 45.3228,132.48 45.5354,132.48 
 45.748,132.48 45.9606,132.48 46.1732,132.48 46.3858,132.48 46.5984,132.48 46.811,132.48 47.0236,132.48 47.2362,132.48 47.4488,132.48 47.6614,132.48 
 47.874,132.48 48.0866,132.48 48.2992,132.48 48.5118,132.48 48.7244,132.48 48.937,132.48 49.1496,132.48 49.3622,132.48 49.5748,132.48 49.7874,132.48 
 50,132.48 50.2126,132.48 50.4252,132.48 50.6378,132.48 50.8504,132.48 51.063,132.48 51.2756,132.48 51.4882,132.48 51.7008,132.48 51.9134,132.48 
 52.126,132.48 52.3386,132.48 52.5512,132.48 52.7638,132.48 52.9764,132.48 53.189,132.48 53.4016,132.48 53.6142,132.48 53.8268,132.48 54.0394,132.48 
 54.252,132.48 54.4646,132.48 54.6772,132.48 54.8898,132.48 55.1024,132.48 55.315,132.48 55.5276,132.48 55.7402,132.48 55.9528,132.48 56.1654,132.48 
 56.378,132.48 56.5906,132.48 56.8031,132.48 57.0157,132.48 57.2283,132.48 57.4409,132.48 57.6535,132.48 57.8661,132.48 58.0787,132.48 58.2913,132.48 
 58.5039,132.48 58.7165,132.48 58.9291,132.48 59.1417,132.48 59.3543,132.48 59.5669,132.48 59.7795,132.48 59.9921,132.48 60.2047,132.48 60.4173,132.48 
 60.6299,132.48 60.8425,132.48 61.0551,132.48 61.2677,132.48 61.4803,132.48 61.6929,132.48 61.9055,132.48 62.1181,132.48 62.3307,132.48 62.5433,132.48 
 62.7559,132.48 62.9685,132.48 63.1811,132.48 63.3937,132.48 63.6063,132.48 63.8189,132.48 64.0315,132.48 64.2441,132.48 64.4567,132.48 64.6693,132.48 
 64.8819,132.48 65.0945,132.48 65.3071,132.48 65.5197,132.48 65.7323,132.48 65.9449,132.48 66.1575,132.48 66.3701,132.48 66.5827,132.48 66.7953,132.48 
 67.0079,132.48 67.2205,132.48 67.4331,132.48 67.6457,132.48 67.8583,132.48 68.0709,132.48 68.2835,132.48 68.4961,132.48 68.7087,132.48 68.9213,132.48 
 69.1339,132.48 69.3465,132.48 69.5591,132.48 69.7717,132.48 69.9843,132.48 70.1969,132.48 70.4094,132.48 70.622,132.48 70.8346,132.48 71.0472,132.48 
 71.2598,132.48 71.4724,132.48 71.685,132.48 71.8976,132.48 72.1102,132.48 72.3228,132.48 72.5354,132.48 72.748,132.48 72.9606,132.48 73.1732,132.48 
 73.3858,132.48 73.5984,132.48 73.811,132.48 74.0236,132.48 74.2362,132.48 74.4488,132.48 74.6614,132.48 74.874,132.48 75.0866,132.48 75.2992,132.48 
 75.5118,132.48 75.7244,132.48 75.937,132.48 76.1496,132.48 76.3622,132.48 76.5748,132.48 76.7874,132.48 77,132.48 77.2126,132.48 77.4252,132.48 
 77.6378,132.48 77.8504,132.48 78.063,132.48 78.2756,132.48 78.4882,132.48 78.7008,132.48 78.9134,132.48 79.126,132.48 79.3386,132.48 79.5512,132.48 
 79.7638,132.48 79.9764,132.48 80.189,132.48 80.4016,132.48 80.6142,132.48 80.8268,132.48 81.0394,132.48 81.252,132.48 81.4646,132.48 81.6772,132.48 
 81.8898,129.53 82.1024,126.58 82.315,123.629 82.5276,120.679 82.7402,117.729 82.9528,114.778 83.1654,111.828 83.378,108.878 83.5906,105.928 83.8031,102.977 
 84.0157,100.027 84.2283,97.0767 84.4409,94.1264 84.6535,91.1761 84.8661,88.2258 85.0787,85.2755 85.2913,82.3252 85.5039,79.3749 85.7165,76.4246 85.9291,73.4743 
 86.1417,70.524 86.3543,67.5737 86.5669,64.6234 86.7795,61.6731 86.9921,58.7228 87.2047,55.7725 87.4173,52.8222 87.6299,49.8719 87.8425,46.9216 88.0551,43.9714 
 88.2677,41.0211 88.4803,38.0708 88.6929,35.1205 88.9055,32.1702 89.1181,29.2199 89.3307,26.2696 89.5433,23.3193 89.7559,20.369 89.9685,17.41

In [8]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=1,linecolor="blue")
plot!(pltsx,pltsy,st=:scatter)
plot!(sx,sy,st=:scatter)
a=[0.15 0.15]
b=[0.19 0.19]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
<polyline clip-path="url(#clip7102)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 252.517,219.943 252.517,219.943 252.517,219.943 252.517,219.943 252.517,219.943 252.517,219.943 252.517,219.942 252.517,219.942 252.516,219.942 252.516,219.942 
 252.515,219.941 252.514,219.941 252.512,219.94 252.511,219.939 252.509,219.938 252.506,219.936 252.503,219.934 252.5,219.932 252.496,219.93 252.492,219.927 
 252.487,219.923 252.481,219.92 252.475,219.916 252.469,219.911 252.461,219.906 252.453,219.9 252.445,219.894 252.435,219.887 252.425,219.88 252.414,219.872 
 252.403,219.864 252.391,219.855 252.377,219.845 252.364,219.835 252.349,219.824 252.334,219.813 252.318,219.801 252.301,219.788 252.283,219.774 252.265,219.76 
 252.246,219.745 252.226,219.729 252.205,219.713 252.183,219.696 252.161,219.678 252.138,219.66 252.114,219.64 252.089,219.62 252.064,219.6 252.038,219.578 
 252.011,219.556 251.983,219.533 251.954,219.509 251.925,219.484 251.895,219.459 251.864,219.433 251.833,219.406 251.8,219.378 251.767,219.349 251.733,219.32 
 251.699,219.289 251.663,219.258 251.627,219.226 251.59,219.193 251.553,219.159 251.514,219.125 251.475,219.089 251.435,219.053 251.395,219.016 251.354,218.978 
 251.311,218.939 251.269,218.899 251.225,218.858 251.181,218.816 251.136,218.773 251.09,218.73 251.043,218.685 250.996,218.639 250.948,218.593 250.899,218.545 
 250.849,218.497 250.799,218.447 250.748,218.397 250.696,218.345 250.643,218.292 250.589,218.238 250.535,218.184 250.48,218.128 250.424,218.071 250.367,218.013 
 250.309,217.953 250.251,217.893 250.191,217.832 250.131,217.769 250.07,217.705 250.008,217.64 249.945,217.574 249.882,217.506 249.817,217.437 249.751,217.367 
 249.685,217.296 249.617,217.223 249.549,217.149 249.48,217.074 249.409,216.997 249.338,216.919 249.266,216.84 249.192,216.759 249.118,216.676 249.042,216.593 
 248.966,216.507 248.888,216.421 248.809,216.332 248.729,216.242 248.648,216.151 248.566,216.058 248.482,215.963 248.398,215.867 248.312,215.769 248.225,215.669 
 248.136,215.568 248.046,215.464 247.955,215.36 247.863,215.253 247.769,215.144 247.674,215.034 247.578,214.921 247.48,214.807 247.381,214.691 247.28,214.572 
 247.177,214.452 247.074,214.33 246.968,214.205 246.861,214.079 246.753,213.95 246.642,213.819 246.53,213.686 246.417,213.55 246.302,213.413 246.185,213.273 
 246.066,213.13 245.945,212.985 245.822,212.838 245.698,212.688 245.572,212.536 245.443,212.38 245.313,212.223 245.181,212.062 245.046,211.899 244.91,211.733 
 244.771,211.564 244.63,211.392 244.487,211.218 244.341,211.04 244.193,210.859 244.043,210.675 243.89,210.488 243.735,210.297 243.577,210.103 243.417,209.906 
 243.254,209.705 243.088,209.501 242.92,209.293 242.748,209.081 242.574,208.866 242.397,208.646 242.217,208.423 242.034,208.196 241.847,207.965 241.658,207.729 
 241.465,207.49 241.269,207.246 241.069,206.998 240.866,206.745 240.66,206.488 240.45,206.226 240.237,205.959 240.019,205.688 239.798,205.412 239.574,205.131 
 239.345,204.845 239.113,204.554 238.876,204.257 238.636,203.956 238.392,203.65 238.144,203.338 237.891,203.022 237.635,202.7 237.375,202.372 237.11,202.04 
 236.841,201.702 236.569,201.358 236.292,201.01 236.011,200.656 235.726,200.297 235.437,199.933 235.145,199.563 234.848,199.189 234.548,198.81 234.243,198.425 
 233.936,198.036 233.624,197.643 233.31,197.245 232.992,196.842 232.67,196.435 232.346,196.025 232.019,195.61 231.689,195.192 231.357,194.77 231.022,194.345 
 230.684,193.917 230.345,193.486 230.004,193.052 229.661,192.616 229.317,192.178 228.971,191.738 228.625,191.296 228.277,190.853 227.929,190.408 227.58,189.963 
 227.231,189.518 226.883,189.072 226.534,188.626 

In [23]:
function trj_generator(sx,xv,tf,te,dt,legnum)#xの遊脚軌道
    #
    #条件ベクトル
    xf=sx[legnum,tf]
    xe=sx[legnum,te]
    vf=xv[tf]*-1
    ve=xv[te]*-1
    A=[xf; xe; vf; ve]
    println("stepf=",tf," stepe=",te," xf=",xf," xe=",xe,"vf=",vf,"ve=",ve)
    tf=round(tf*dt,6) #時間に戻す
    te=round(te*dt,6) #第6位で切り捨て
        
    B=[tf^3 tf^2 tf 1
        te^3 te^2 te 1
        3tf^2 2tf 1 0
        3te^2 2te 1 0]

    C=inv(B)*A

    #dt=0.01 #離散時間
    t=[i for i in tf:dt:te] #描画時間
    x=C[1]*.^(t,3)+C[2]*.^(t,2)+C[3]*t+C[4] #位置
    v=3*C[1]*.^(t,2)+2*C[2]*t+C[3] #速度
    a=6*C[1]*t+2*C[2]
    
    return x
    
end
function trj_generator_z(zh,zf,ze,ve,tf,te,dt)
    #support time 0.5
    tf=round(tf*dt,7) #時間に戻す
    te=round(te*dt,7) #第6位で切り捨て
    th=round((te-tf)/2+tf,7)
    
    zh=zf+zh
    vf=zero(ve)
   
    A_z=[zh; zf; ze; vf; ve]

    B_z=[th^4 th^3 th^2 th 1
        tf^4 tf^3 tf^2 tf 1
        te^4 te^3 te^2 te 1
        4tf^3 3tf^2 2tf 1 0
        4te^3 3te^2 2te 1 0]
    C_z=inv(B_z)*A_z
    
    t=[i for i in tf:dt:te]; #描画時間
    z=C_z[1]*.^(t,4)+C_z[2]*.^(t,3)+C_z[3]*.^(t,2)+C_z[4]*t+C_z[5] #位置
    v_z=4*C_z[1]*.^(t,3)+3*C_z[2]*.^(t,2)+2*C_z[3]*t+C_z[4] #速度
    a_z=12*C_z[1]*.^(t,2)+6*C_z[2]*t+2*C_z[3]; #加速度
    return z
end

In [31]:
#脚先位置を重心の座標系に変換する
com_trjx=copy(x_plot_log) #重心位置
com_trjy=copy(y_plot_log)

sx_from_com=zeros(3,length(com_trjx))
sy_from_com=zeros(3,length(com_trjy))   #重心からみた支持脚位置
sz_from_com=[-zc for i in 1:length(com_trjx)*3]
sz_from_com=reshape(sz_from_com,3,length(com_trjx))

sx_from_w=reshape(real_sx,3,vpoint_sum)
sy_from_w=reshape(real_sy,3,vpoint_sum)
vz_end=0.0 #遊脚が接地する速度

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum) 
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
        end
    else
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
            
        end
        if(i!=1)
            sx_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sx_from_com,xv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sy_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sy_from_com,yv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sz_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator_z(0.05,-zc,-zc,vz_end,1+step,Tsup+step,dt)
        end
        step=step+Tsup
    
        for s in 1:Tdl
            sx_from_com[:,s+step]=sx_from_w[:,i]-com_trjx[s+step]
            sy_from_com[:,s+step]=sy_from_w[:,i]-com_trjy[s+step]
        end
        step=step+Tdl
    end
end
;
#-1が使われている可能性あり

stepf=241 stepe=440 xf=0.043608200517266396 xe=0.033558223201398674vf=0.04461079504364818ve=-0.11653419711097819
stepf=241 stepe=440 xf=-0.07821512635937472 xe=-0.07876529810081631vf=-0.07062298288345122ve=0.06856038798771949
stepf=481 stepe=680 xf=-0.08363419392642728 xe=-0.08479424119727096vf=-0.12152617354037708ve=0.1163153661038315
stepf=481 stepe=680 xf=-0.010811869541207608 xe=0.009907990110606342vf=0.07478497997097149ve=0.0693978652319616
stepf=721 stepe=920 xf=0.03255333695175047 xe=0.050981567959386194vf=0.12486579123696366ve=0.0019178182809678626
stepf=721 stepe=920 xf=0.0776268155177176 xe=0.06847174890678531vf=0.06923384333040777ve=-0.1353785527079392
stepf=961 stepe=1160 xf=0.05095214169381204 xe=0.033807579403872565vf=-0.002487441742431101ve=-0.11820072759022335
stepf=961 stepe=1160 xf=-0.06700971121124162 xe=-0.07838739922455105vf=-0.14272671790405875ve=0.0660295801419756
stepf=1201 stepe=1400 xf=-0.08350813556090875 xe=-0.0847892664902267vf=-0.12236153503644867ve=0.1162

In [29]:
plot(sx_from_com[1,1:500],st=:scatter)   #241 440   961 1160     1681  1880 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.015 
 
 
 0.020 
 
 
 0.025 
 
 
 0.030 
 
 
 0.035 
 
 
 0.040 
 
 
 0.045 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [11]:
struct Struct1
    x::Float64
    y::Float64
    yaw::Float64
end

LoadError: syntax: extra token "Struct1" after end of expression